In [1]:
import import_ipynb
import sys
import torch
from torch import nn
from torch import optim

if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

%cd /home/ankit/Commitments/earthquake-prediction/

from preprocessing import *
from training import *
from postprocessing import *
from testing import *

%cd /home/ankit/Commitments/earthquake-prediction/Mainland China/

/home/ankit/Commitments/earthquake-prediction
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from training.ipynb
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 90])
importing Jupyter notebook from postprocessing.ipynb
importing Jupyter notebook from testing.ipynb
/home/ankit/Commitments/earthquake-prediction/Mainland China


In [4]:
# SETTINGS D

data_file = "data.csv"
time_parser = time_from_string_usgs
mag_range = None

rows = 3
cols = 3
total_params = rows*cols
time_intervals = 690
train_size = 0.8
lookback_window = 30

In [5]:
earthquakes = extract_from_csv(data_file, time_parser, mag_range)
cells = Spatio_Temporal_Segmentation(rows, cols, time_intervals, earthquakes)

print(cells[0][0][0])

No earthquakes were recorded in this cell.


In [6]:
generator = TensorGen(cells)

eventtensor = generator.eventNoEvent()
freqtensor = generator.freq()
minmagtensor = generator.min_magnitude()
maxmagtensor = generator.max_magnitude()

if torch.cuda.is_available:
    eventtensor = eventtensor.cuda()
    freqtensor = freqtensor.cuda()
    minmagtensor = minmagtensor.cuda()
    maxmagtensor = maxmagtensor.cuda()
    
print(eventtensor.size())
print(freqtensor.size())
print(minmagtensor.size())
print(maxmagtensor.size())

torch.Size([690, 1, 9])
torch.Size([690, 1, 9])
torch.Size([690, 1, 9])
torch.Size([690, 1, 9])


<strong>TRAINING EVENT PREDICTOR</strong>

In [30]:
#SETTINGS M1

hidden_parameters = 40
linear_parameters = 30
num_layers = 5

In [31]:
model = LSTM(total_params, hidden_parameters, linear_parameters, total_params, num_layers=num_layers, activation=nn.Tanh())

if torch.cuda.is_available():
    model = model.cuda()
    
# out = model(eventtensor)
# print(out)

In [32]:
# SETTINGS T1
pos_weight = torch.tensor([1])
pos_weight = pos_weight.cuda()
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
loss_function = nn.MSELoss()
n_epochs = 100

In [33]:
optimizer = optim.Adagrad(model.parameters(), lr=0.01)

# tqdm._instances.clear()
train(model, eventtensor, optimizer, loss_function, n_epochs=1)

100%|██████████| 640/640 [00:10<00:00, 62.12it/s] 

Epoch 1 completed. 
Training Loss: 0.157
Test Loss: 0.162


In [34]:
evaluate(data=eventtensor, model=model, train_size=0.8, lookback_window=lookback_window, threshold=0.5)

100%|██████████| 660/660 [00:03<00:00, 179.32it/s]


Evaluation results for training dataset:
4698 evaluations in total.
2211/2479 cells with earthquake activity were correctly identified. (89.19 %)
776/2219 cells with no earthquake activity were correctly identified. (34.97 %)
1443/3654 positives turned out to be false.
268/1044 negatives turned out to be false.
Preicision: 0.605
Recall: 0.892
Overall accuracy: 63.58 %
Evaluation results for training dataset:
1242 evaluations in total.
764/893 cells with earthquake activity were correctly identified. (85.55 %)
147/349 cells with no earthquake activity were correctly identified. (42.12 %)
202/966 positives turned out to be false.
129/276 negatives turned out to be false.
Preicision: 0.791
Recall: 0.856
Overall accuracy: 73.35 %
